## 1. Importaciones

In [ ]:
# Paquetes necesarios
import pandas as pd
import numpy as np

import requests as rd
import time
import json
import sys
import seaborn as sb
import math

import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import IPython.display
from IPython.display import display, clear_output

from matplotlib import pyplot as plt
%matplotlib inline

In [1]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google'

## Parámetros y constantes

### Parámetros

In [ ]:
# Variables parámetricas
# Introducir el año para el que se quieren sacar los reportes
AÑO = widgets.Dropdown(
    options= [2018,2019, 2020, 2021, 2022],
    value = 2019,
    description = 'Año',
)

def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        print("changed to %s" % change['new'])

AÑO.observe(on_change)

display(AÑO)

Dropdown(description='Año', index=1, options=(2018, 2019, 2020, 2021, 2022), value=2019)

changed to 2018


### Cargar el dataset de gastos

In [ ]:
# Constantes
DIR_DATASETS = "/content/drive/MyDrive/Proyectos/CINTRA/Datos"

#nombres carpetas y ficheros
CARPETA_DATASETS = f"datos_{AÑO.value}"

sub_carpeta_DATASET_GASTOS = f"EPFgastos_{AÑO.value}"
sub_carpeta_DATASET_HOGAR = f"EPFhogar_{AÑO.value}"
sub_carpeta_DATASET_mHOGAR = f"EPFmhogar_{AÑO.value}"

NOMBRE_DATASET_GASTOS = f"EPFgastos_{AÑO.value}.csv"
NOMBRE_DATASET_HOGAR = f"EPFhogar_{AÑO.value}.csv"
NOMBRE_DATASET_mHOGAR = f"EPFmhogar_{AÑO.value}.csv"

#importar datos
df_EPFgastos_ = pd.read_csv(f'{DIR_DATASETS}/{CARPETA_DATASETS}/{sub_carpeta_DATASET_GASTOS}/CSV/{NOMBRE_DATASET_GASTOS}', sep='\t', low_memory=False)
df_EPFhogar_ = pd.read_csv(f'{DIR_DATASETS}/{CARPETA_DATASETS}/{sub_carpeta_DATASET_HOGAR}/CSV/{NOMBRE_DATASET_HOGAR}', sep='\t', low_memory=False)
df_EPFmhogar_ = pd.read_csv(f'{DIR_DATASETS}/{CARPETA_DATASETS}/{sub_carpeta_DATASET_mHOGAR}/CSV/{NOMBRE_DATASET_mHOGAR}', sep='\t', low_memory=False)


In [ ]:
df_EPFhogar_

,ANOENC,NUMERO,CCAA,NUTS1,CAPROV,TAMAMU,DENSIDAD,CLAVE,CLATEO,FACTOR,...,FUENPRIN,FUENPRINRED,IMPEXAC,INTERIN,NUMPERI,COMIMH,COMISD,COMIHU,COMIINV,COMITOT
0,2018,1,2,2,1,1,1,2,2,422.866646,...,2,2,2324,5,2,54,0,0,0,54
1,2018,2,9,5,6,4,2,1,1,2051.951724,...,2,2,2682,6,2,64,0,0,0,64
2,2018,3,13,3,1,1,1,1,1,1349.187753,...,3,3,3240,7,2,52,0,0,0,52
3,2018,4,10,5,1,1,1,1,1,1200.012995,...,3,3,746,2,1,28,0,0,0,28
4,2018,5,1,6,6,3,2,1,1,887.660816,...,3,3,3150,7,2,54,0,0,2,56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21390,2018,21391,5,7,6,1,1,1,1,956.731165,...,3,3,733,2,1,28,0,0,10,38
21391,2018,21392,9,5,6,3,1,2,2,1230.268422,...,3,3,3845,7,3,60,1,0,28,89
21392,2018,21393,2,2,1,1,1,2,2,650.277852,...,2,2,1267,3,1,82,0,0,0,82
21393,2018,21394,3,1,1,1,1,2,2,483.812199,...,2,2,1200,3,1,92,0,0,0,92


# Evaluación de los datos de la  Encuesta de Presupuestos Familiares. Limpieza y transformación de datos.

## Objetivos
Este Notebook tiene como objetivo la evaluación del impacto de una posible tarificación en las rentas de la población española.
Para ello se analizan los datos de renta, gastos y características del hogar tomados de la Encuesta de presupuestos familiares del INE.
Se presentan los principales resultados de la evaluación de los datos de la encuesta de presupuestos familiares.

## Datos de la peninsula

In [ ]:
# eliminar hogares que no pertenecen a la peninsula
islas_territorios_ultra = [4, 5, 18, 19]
df_EPFhogarPeninsula_ = (df_EPFhogar_.query("CCAA not in @islas_territorios_ultra"))

In [ ]:
hogares_peninsula_ = list(df_EPFhogarPeninsula_.NUMERO.unique())
total_hogares_peninsula_ = len(hogares_peninsula_)

## Tabla de gastos de los hogares

### Inspección inicial de la tabla de gastos de los hogares: generalidades del conjunto de datos.

Esta sección muestra el análisis exploratorio de la tabla de gastos.

In [ ]:
#inspeccion visual
df_EPFgastos_.head()

,ANOENC,NUMERO,CODIGO,GASTO,PORCENDES,PORCENIMP,CANTIDAD,GASTOMON,GASTNOM1,GASTNOM2,GASTNOM3,GASTNOM4,GASTNOM5,FACTOR
0,2021,1,01113,31051.14,0.0,0.0,18326.47,31051.14,NaN,NaN,NaN,NaN,NaN,585.77697
1,2021,1,01136,117292.44,0.0,0.0,12217.64,117292.44,NaN,NaN,NaN,NaN,NaN,585.77697
2,2021,1,01163,68996.09,0.0,0.0,51924.99,68996.09,NaN,NaN,NaN,NaN,NaN,585.77697
3,2021,1,01167,86934.65,0.0,0.0,137448.49,86934.65,NaN,NaN,NaN,NaN,NaN,585.77697
4,2021,1,0117A,139711.81,0.0,0.0,103849.97,139711.81,NaN,NaN,NaN,NaN,NaN,585.77697


In [ ]:
# se verifica el no. de columnas y filas de el dataset
AÑO_ = df_EPFgastos_.ANOENC.unique()[0]
observaciones = df_EPFgastos_.shape[0]
campos =  df_EPFgastos_.shape[1]

In [ ]:
#verificación de la cantidad de familias en el dataset
total_hogares = len(df_EPFgastos_['NUMERO'].unique())

In [ ]:
print(f"La tabla original de gastos del año {AÑO_} esta compuesta por {observaciones:,} observaciones y {campos} columnas.")

La tabla original de gastos del año 2021 esta compuesta por 1,488,799 observaciones y 14 columnas.


In [ ]:
print(f"Esas {observaciones:,} observaciones contenían datos de {total_hogares:,} familias.")

Esas 1,488,799 observaciones contenían datos de 19,394 familias.


In [ ]:
#verificacion del total de columas de la tabla
columnas_ = str(list(df_EPFgastos_.columns)).replace('[','(').replace(']',')')
print(f"Esta tabla contiene los siguientes campos: {columnas_}.")

Esta tabla contiene los siguientes campos: ('ANOENC', 'NUMERO', 'CODIGO', 'GASTO', 'PORCENDES', 'PORCENIMP', 'CANTIDAD', 'GASTOMON', 'GASTNOM1', 'GASTNOM2', 'GASTNOM3', 'GASTNOM4', 'GASTNOM5', 'FACTOR').


In [ ]:
# estadisticas de todas las columnas
df_EPFgastos_.describe().apply(lambda s: s.apply(lambda x: format(x, 'f')))

,ANOENC,NUMERO,GASTO,PORCENDES,PORCENIMP,CANTIDAD,GASTOMON,GASTNOM1,GASTNOM2,GASTNOM3,GASTNOM4,GASTNOM5,FACTOR
count,1488799.000000,1488799.000000,1488799.000000,1488799.000000,1488799.000000,590133.000000,1488797.000000,2447.000000,1530.000000,1895.000000,19749.000000,0.000000,1488799.000000
mean,2021.000000,9679.386243,370580.949039,37.625300,14.404629,258994.555866,285586.451390,614732.243658,185281.258614,956202.738602,6225153.676742,nan,983.451849
std,0.000000,5587.629947,1638910.071170,47.563219,34.501169,1270695.138794,1288340.028931,3027980.384039,767557.184837,3138617.132186,6322289.243493,nan,736.802477
min,2021.000000,1.000000,0.010000,0.000000,0.000000,10.010000,0.000000,0.000000,0.780000,92.030000,84530.720000,nan,74.113862
25%,2021.000000,4839.000000,5591.385000,0.000000,0.000000,8155.760000,4469.960000,71902.035000,11651.937500,8113.860000,2542857.330000,nan,457.225187
50%,2021.000000,9692.000000,51609.080000,0.000000,0.000000,27116.200000,47052.940000,172347.040000,48621.745000,101021.960000,4230136.940000,nan,782.325458
75%,2021.000000,14482.000000,215188.635000,100.000000,0.000000,92687.740000,197338.820000,395628.740000,143224.882500,750210.300000,7803855.910000,nan,1331.782317
max,2021.000000,19394.000000,279792785.140000,100.000000,100.000000,111814792.220000,279792785.140000,102036684.650000,16373936.310000,57075373.960000,244465598.020000,nan,9423.128026


In [ ]:
# verifico el número de valores unicos para cada variable
df_EPFgastos_.nunique(axis=0)

ANOENC             1
NUMERO         19394
CODIGO           358
GASTO        1273612
PORCENDES       9799
PORCENIMP       8382
CANTIDAD      394341
GASTOMON     1251383
GASTNOM1        2368
GASTNOM2        1480
GASTNOM3        1795
GASTNOM4       19508
GASTNOM5           0
FACTOR         16627
dtype: int64

In [ ]:
print(f"De esas {total_hogares:,} familias {total_hogares_peninsula_:,} son de la peninsula.")

De esas 19,394 familias 17,835 son de la peninsula.


### Limpieza de los datos

In [ ]:
#verificacion de valores nulos
df_EPFgastos_.isnull().sum()

ANOENC             0
NUMERO             0
CODIGO             0
GASTO              0
PORCENDES          0
PORCENIMP          0
CANTIDAD      898666
GASTOMON           2
GASTNOM1     1486352
GASTNOM2     1487269
GASTNOM3     1486904
GASTNOM4     1469050
GASTNOM5     1488799
FACTOR             0
dtype: int64

No existen valores nulos en las columnas de interés. Pero varias columnas tienen una gran cantidad de valores nulos.

Se verifican las columnas cuyos valoers nulos suponen más de un 90% y estas columnsa son eliminadas de la tabla.

In [ ]:
df_EPFgastos_.isnull()['GASTNOM5'].mean()

1.0

### Selección de hogares de interes (peninsula)

In [ ]:
df_EPFgastosPeninsula_ = (df_EPFgastos_.query("NUMERO in @hogares_peninsula_"))

#### Selección de columnas de interes

In [ ]:
# seleccion de columnas de interés, incluyo la cantidad PERO si el gasto no es monetario (especie o autoconsumo) puede tener una cantidad y no un gasto monetario
columnas_gasto = ['ANOENC','NUMERO','CODIGO','GASTOMON','FACTOR']
columnas_cantidad = ['ANOENC','NUMERO','CODIGO','CANTIDAD','GASTOMON','FACTOR']

In [ ]:
# importación de archivo con columnas de interes
df_EPFgastos_pre = df_EPFgastosPeninsula_[columnas_cantidad]

In [ ]:
df_EPFgastos_pre.describe().apply(lambda s: s.apply(lambda x: format(x, 'f')))

,ANOENC,NUMERO,CANTIDAD,GASTOMON,FACTOR
count,1345109.000000,1345109.000000,537031.000000,1345107.000000,1345109.000000
mean,2021.000000,9700.682830,262056.086374,294268.564298,992.553326
std,0.000000,5579.201363,1271601.195721,1317856.887834,742.288217
min,2021.000000,1.000000,13.310000,0.000000,90.795799
25%,2021.000000,4875.000000,8467.035000,4793.510000,455.504194
50%,2021.000000,9715.000000,27932.300000,50017.860000,786.554423
75%,2021.000000,14476.000000,94975.240000,205741.740000,1359.349189
max,2021.000000,19394.000000,111814792.220000,279792785.140000,9423.128026


### Selección de gastos de interes

Se seleccionan aquellos codigos con relevancia para el estudio. Codigos de gasto en transporte y gastos indispensables.

In [ ]:
# lista con codigos de gastos relevantes para el estudio, aquí se incluye el transporte terrestre, aéreo, marítimo, etc.
# estos codigos se seleccionaron de toda la lista que provee el INE considerando aquellos que indiquen posesión de vehiculos privados
codigos_transporte_general = ['04123','04123','04124','07111','07112','07120','07130','07211','07212','07213','07221','07222','07223','07224','07230','07241','07242','07243','07244','07245','07311','07312','07313','07321','07322','07323','07324','07331','07332','07341','07342','07350','07361','07362']

# estos son codigos de gastos necesarios
gastos_innecesarios = ['01223','02111','02112','02132','02134','02201','02202','02203','04121','04124','04129','04221','04223','04412','04422','04432','04442','04444','04449','04512','04522','04524','04532','04542','04549','05112','05621','05622','07111','07112','07324','07332','07341','07342','07361','07362','07369','08203','09113','09119','09121','09122','09123','09132','09141','09142','09210','09221','09222','09230','09311','09322','09323','09331','09332','09341','09342','09350','09411','09421','09423','09430','09521','09522','09601','09602','11112','11113','11114','11201','11202','11204','12113','12311','12312','12313','12321','12522','12530','12542'
]

codigos_no_coche = ['01111','01112','01113','01114','01115','01116','01117','01118','01121','01122','01123','01124','01125','01126','01127','01128','01131','01132','01133','01134','01135','01136','01141','01142','01143','01144','01145','01146','01147','01151','01152','01153','01154','01155','01160','01161','01162','01163','01164','01165','01166','01167','01168','01169','01171','01172','01173','01174','01175','01176','01177','01178','01179','0117A','0117B','0117C','01181','01182','01183','01184','01185','01186','01191','01192','01193','01194','01195','01199','01211','01212','01213','01214','01221','01222','01223','01224','01225','02111','02112','02121','02122','02123','02124','02131','02132','02133','02134','02201','02202','02203','03110','03121','03122','03123','03131','03132','03141','03142','03211','03212','03213','03220','04110','04121','04122','04129','04210','04221','04222','04223','04310','04321','04322','04323','04324','04325','04329','04411','04412','04421','04422','04431','04432','04441','04442','04443','04444','04448','04449','04511','04512','04521','04522','04523','04524','04531','04532','04541','04542','04548','04549','05111','05112','05113','05114','05115','05121','05122','05123','05130','05201','05202','05203','05204','05209','05311','05312','05313','05314','05315','05319','05321','05322','05323','05324','05329','05330','05401','05402','05403','05404','05511','05512','05521','05522','05523','05611','05612','05621','05622','05623','05624','05629','06111','06112','06121','06129','06131','06132','06133','06139','06211','06212','06220','06231','06232','06239','06300','07311','07312','07313','07321','07322','07323','07324','07331','07332','07341','07342','07350','07361','07362','07369','08101','08109','08201','08202','08203','08204','08301','08302','08303','08304','08309','09111','09112','09113','09119','09121','09122','09123','09131','09132','09133','09134','09141','09142','09149','09150','09210','09221','09222','09230','09311','09312','09321','09322','09323','09331','09332','09341','09342','09350','09411','09412','09421','09422','09423','09424','09425','09429','09430','09511','09512','09513','09514','09521','09522','09530','09541','09549','09601','09602','10111','10112','10113','10114','10121','10122','10211','10212','10213','10214','10221','10222','10223','10224','10225','10301','10302','10303','10304','10305','10306','10400','11111','11112','11113','11114','11115','11116','11121','11122','11130','11201','11202','11203','11204','12111','12112','12113','12121','12122','12131','12132','12311','12312','12313','12321','12322','12323','12328','12329','12401','12402','12403','12404','12521','12522','12530','12541','12542','12551','12552','12559','12620','12701','12702','12703','12704','12810','12820'
]
#conteo de codigos relacionados con el transporte
len(codigos_transporte_general)
len(gastos_innecesarios)

80

In [ ]:
#Creación de un nuevo marco de datos basado en la seleccion de filas con gastos de transporte
df_EPFgastos_transporte = df_EPFgastos_pre.loc[df_EPFgastos_pre['CODIGO'].isin(codigos_transporte_general)]

In [ ]:
df_EPFgastos_necesarios = (df_EPFgastos_pre.query("CODIGO not in @gastos_innecesarios"))

In [ ]:
# verificacion de cantidad de familias con gastos de transporte
hogares_gastos_transporte = len(df_EPFgastos_transporte['NUMERO'].unique())

In [ ]:
print(f"De {total_hogares_peninsula_} familias residentes en la peninsula solo {hogares_gastos_transporte} presentaron gastos de transporte. Esto incluye compra de billetes de avión, tren, gasto en carburante, etc.")

De 17835 familias residentes en la peninsula solo 14877 presentaron gastos de transporte. Esto incluye compra de billetes de avión, tren, gasto en carburante, etc.


### EDA inicial de la tabla con hogares que tienen gastos que indican _car ownership_: **De interés**

De aquellas con gastos de transporte se seleccionaron aquellas con gastos en carburantes (Gasóleo o Gasolina).

In [ ]:
# seleccion de codigo con carburantes
# 07221 = Gasóleo
# 07222 = Gasolina
codigos_carburantes = ['07221','07222']

In [ ]:
# seleccion de codigos que indican _car ownership_
codigos_posesion_vehiculoprivado = ['04123','04124','07111','07112','07120','07211','07212','07213','07221','07222','07223','07224','07230','07241','07242','07243','07244','07245']

In [ ]:
#seleccion de hogares con gastos de carburantes
df_EPFgastos_carburante = df_EPFgastos_pre.loc[df_EPFgastos_pre['CODIGO'].isin(codigos_carburantes)]
df_EPFgastos_cocheprivado = df_EPFgastos_pre.loc[df_EPFgastos_pre['CODIGO'].isin(codigos_posesion_vehiculoprivado)]

In [ ]:
# inspeccion familias gastos relacionados a tenencia de coche y gastos en carburante
no_familias_gastoCarburante = len(df_EPFgastos_carburante.NUMERO.unique())
no_familias_gastoCoche = len(df_EPFgastos_cocheprivado.NUMERO.unique())
print(f"De {hogares_gastos_transporte} que presentaron gastos de transporte, {no_familias_gastoCoche} presentaron gastos relacionados con la posesión de coche particular.")
print(f'De esas {no_familias_gastoCoche} con gastos indicativos de posesión de coche, solo {no_familias_gastoCarburante} presentaron gastos de Gasóleo o Gasolina.')

De 14877 que presentaron gastos de transporte, 14324 presentaron gastos relacionados con la posesión de coche particular.
De esas 14324 con gastos indicativos de posesión de coche, solo 9063 presentaron gastos de Gasóleo o Gasolina.


### Data Wrangling
1. Cambio de la forma de la tabla de formato *long* a formato *wide*

In [ ]:
# obtención gasto total familiar para comprobación posterior
verificacion_coche = df_EPFgastos_cocheprivado.groupby('NUMERO').GASTOMON.sum().to_frame().reset_index()
verificacion_carburante = df_EPFgastos_carburante.groupby('NUMERO').GASTOMON.sum().to_frame().reset_index()
verificacion_coche = verificacion_coche.set_index('NUMERO')
verificacion_carburante = verificacion_carburante.set_index('NUMERO')

In [ ]:
# cambio de forma de la tabla todo junto APPROACH 2
# usando pivot_table normal pero por separado para cada columna y luego agregando la columna factor; tambien con multi-index
cantidad = df_EPFgastos_carburante.pivot(index='NUMERO',columns='CODIGO',values='CANTIDAD')
gasto_mon = df_EPFgastos_carburante.pivot(index='NUMERO',columns='CODIGO',values='GASTOMON')

#cambio de nombre de las columnas
cantidad = cantidad.add_prefix('cantidad_')
gasto_mon = gasto_mon.add_prefix('gasto_')

#union de los data frames
df_EPFgastos_carburante_2 = pd.concat([cantidad,gasto_mon], axis=1)

#inclusion de la columna factor en el subset de datos
df_EPFgastos_carburante_factor =  df_EPFgastos_carburante.copy()
df_EPFgastos_carburante_temp = df_EPFgastos_carburante_factor.drop_duplicates(subset=['NUMERO'], keep='first')
df_EPFgastos_carburante_2['FACTOR'] = np.where(df_EPFgastos_carburante_2.index == df_EPFgastos_carburante_temp['NUMERO'], df_EPFgastos_carburante_temp['FACTOR'], 0)

In [ ]:
# verificación de que no existan familias sin FACTOR
sum(df_EPFgastos_carburante_2['FACTOR'].isnull())

0

In [ ]:
# Verificación de que el gasto sea el mismo despues del reshape
# reemplazar nan con 0's
df_EPFgastos_carburante_2.fillna(0, inplace = True)
verificacion_carburante.fillna(0, inplace = True)
df_EPFgastos_carburante_2['_gasto_total_carbu'] = df_EPFgastos_carburante_2['gasto_07221'] + df_EPFgastos_carburante_2['gasto_07222']
x = df_EPFgastos_carburante_2['_gasto_total_carbu'] - verificacion_carburante['GASTOMON']
sum(x)

0.0

In [ ]:
# cambio de formato de la tabla todo junto APPROACH 2
# usando pivot_table normal pero por separado para cada columna y luego agregando la columna factor; tambien con multi-index
# excluí la columna cantidad porque sería demasiado larga la tabla
gasto_mon = df_EPFgastos_cocheprivado.pivot(index='NUMERO', columns='CODIGO', values='GASTOMON')

# cambio de nombre de las columnas
df_EPFgastos_cocheprivado_2 = gasto_mon.add_prefix('gasto_')

#inclusion de la columna factor en el subset de datos
df_EPFgastos_cocheprivado_factor =  df_EPFgastos_cocheprivado.copy()
df_EPFgastos_carburante_temp = df_EPFgastos_cocheprivado.drop_duplicates(subset=['NUMERO'], keep='first')
df_EPFgastos_cocheprivado_2['FACTOR'] = np.where(df_EPFgastos_cocheprivado_2.index == df_EPFgastos_carburante_temp['NUMERO'], df_EPFgastos_carburante_temp['FACTOR'], 0)

In [ ]:
# verificación de que no existan familias sin FACTOR
sum(df_EPFgastos_cocheprivado_2['FACTOR'].isnull())

0

In [ ]:
# Verificación de que el gasto sea el mismo despues del reshape
# reemplazar nan con 0's
df_EPFgastos_cocheprivado_2.fillna(0, inplace = True)
verificacion_coche.fillna(0, inplace = True)
df_EPFgastos_cocheprivado_2['_gasto_total_coche'] = df_EPFgastos_cocheprivado_2[list(df_EPFgastos_cocheprivado_2.filter(regex='gasto_'))].sum(axis=1)
x = df_EPFgastos_cocheprivado_2['_gasto_total_coche'] - verificacion_coche['GASTOMON']
sum(x)

1.124933532992145e-07

Se crean las columnas de gasto total, gasto anual y gasto mensual (en carburante) dividiendo por el gasto factor poblacional.

In [ ]:
# creación de columnas de gasto en carburante
# creacion de la columna gasto total en carburante
df_EPFgastos_cocheprivado_2['_gasto_total_carbu'] = df_EPFgastos_cocheprivado_2['gasto_07221'] + df_EPFgastos_cocheprivado_2['gasto_07222']

# creacion de la columna gasto por año dividiendo entre el factor poblacional
df_EPFgastos_carburante_2['_gasto_total_carbu_año'] = df_EPFgastos_carburante_2['_gasto_total_carbu'].div(df_EPFgastos_carburante_2["FACTOR"].values)
df_EPFgastos_cocheprivado_2['_gasto_total_carbu_año'] = df_EPFgastos_cocheprivado_2['_gasto_total_carbu'].div(df_EPFgastos_cocheprivado_2["FACTOR"].values)

# creacion de la columna gasto por mes dividiendo entre 12
df_EPFgastos_carburante_2['_gasto_total_carbu_mes'] =  df_EPFgastos_carburante_2['_gasto_total_carbu_año'].div(12)
df_EPFgastos_cocheprivado_2['_gasto_total_carbu_mes'] =  df_EPFgastos_cocheprivado_2['_gasto_total_carbu_año'].div(12)

In [ ]:
df_EPFgastos_carburante_2.describe()

CODIGO,cantidad_07221,cantidad_07222,gasto_07221,gasto_07222,FACTOR,_gasto_total_carbu,_gasto_total_carbu_año,_gasto_total_carbu_mes
count,9.063000e+03,9.063000e+03,9.063000e+03,9.063000e+03,9063.000000,9.063000e+03,9063.000000,9063.000000
mean,9.198433e+05,7.513149e+05,1.122470e+06,9.871111e+05,984.415054,2.109581e+06,2242.328535,186.860711
std,1.677917e+06,1.479418e+06,2.059808e+06,1.929678e+06,740.906824,2.601204e+06,1941.949954,161.829163
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,90.795799,0.000000e+00,0.000000,0.000000
25%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,459.769189,5.332339e+05,808.713403,67.392784
50%,2.103508e+05,8.802633e+04,2.602372e+05,1.149408e+05,775.113727,1.303287e+06,1783.184005,148.598667
75%,1.172051e+06,8.716483e+05,1.442332e+06,1.157292e+06,1331.030045,2.731104e+06,3193.887589,266.157299
max,4.761983e+07,1.906966e+07,5.702096e+07,2.769364e+07,9423.128026,5.702096e+07,24431.587081,2035.965590


La celda de arriba muestra que existen personas con gastos de carburante 0 pero aun así fueron incluidos en este dataset. Se inspeccionan esos casos.

In [ ]:
#creacion del dataset con gasto total 0
gasto_vacios = df_EPFgastos_carburante_2.loc[df_EPFgastos_carburante_2['_gasto_total_carbu'] == 0]

In [ ]:
# inspeccion de las columnas
gastos_0 = gasto_vacios.shape[0]

In [ ]:
print(f'Existen {gastos_0} familias con valores de cantidad pero no de gasto. Estos fueron adquiridos con gastos no monetarios (salarios en especie, autoconsumo, etc.).')

Existen 7 familias con valores de cantidad pero no de gasto. Estos fueron adquiridos con gastos no monetarios (salarios en especie, autoconsumo, etc.).


## Tabla de información del hogar: **De interés**

In [ ]:
#codigos de interes la segunda tabla (información hogares)
#Codigos determinados junto a Juan
cols_epfhogar =['ANOENC','NUMERO','CCAA','CAPROV','TAMAMU','DENSIDAD','FACTOR','NMIEMB','TAMANO','NMIEM1','NMIEM2','NMIEM3','NMIEM4','NMIEM5','NMIEM6','NMIEM7','NMIEM8','NMIEM9','NMIEM10','NMIEM11','NMIEM12','NMIEM13','NUMACTI','NUMOCU','NNINOSD','NHIJOSD','UC1','UC2','TIPHOGAR1','TIPHOGAR2','TIPHOGAR3','TIPHOGAR4','TIPHOGAR5','TIPHOGAR6','TIPHOGAR7','TIPHOGAR8','TIPHOGAR9','TIPHOGAR10','TIPHOGAR11','EDADSP','SEXOSP','PAISNACSP','NACIONASP','ECIVILLEGALSP','ESTUDIOSSP','IMPEXACPSP','SECTOR','REGTEN','TIPOEDIF','ZONARES','TIPOCASA','NHABIT','ANNOCON','SUPERF','DISPOSIOV','CAPROP','CAJENA','PENSIO','RENTAS','IMPEXAC','NUMPERI']

In [ ]:
cols_original = len(df_EPFhogar_.columns)

In [ ]:
# Importación de la tabla con columnas preliminarmente de interes
df_EPFhogar_ = pd.read_csv(f'{DIR_DATASETS}/{CARPETA_DATASETS}/{sub_carpeta_DATASET_HOGAR}/CSV/{NOMBRE_DATASET_HOGAR}', sep='\t', usecols=cols_epfhogar)

In [ ]:
# seleccionar hogares peninsula
df_EPFhogarPeninsula_ = (df_EPFhogar_.query("CCAA not in @islas_territorios_ultra"))

In [ ]:
# inspeccion visual
df_EPFhogarPeninsula_.head()

,ANOENC,NUMERO,CCAA,CAPROV,TAMAMU,DENSIDAD,FACTOR,NMIEMB,TAMANO,NMIEM1,...,NHABIT,ANNOCON,SUPERF,DISPOSIOV,CAPROP,CAJENA,PENSIO,RENTAS,IMPEXAC,NUMPERI
0,2021,1,8,6,3,2,585.776970,1,1,1,...,5,1,90,6,6,6,1,6,782,1
1,2021,2,3,6,1,1,627.355782,1,1,1,...,4,6,65,6,6,6,1,6,1283,1
3,2021,4,11,6,3,2,605.490446,1,1,1,...,4,6,70,6,6,6,1,6,1350,1
4,2021,5,10,1,1,1,1748.502373,1,1,1,...,4,6,76,6,6,6,1,6,1206,1
5,2021,6,16,1,1,1,470.264063,4,4,4,...,6,1,115,6,6,1,6,6,7774,2


In [ ]:
cols_peninsula = len(df_EPFhogarPeninsula_.columns)

In [ ]:
print(f'La tabla original de características del hogar estaba compuesta por {cols_original} columnas.')
print(f'La tabla de hogares de la peninsula con las columnas de interes tiene {cols_peninsula}.')

La tabla original de características del hogar estaba compuesta por 188 columnas.
La tabla de hogares de la peninsula con las columnas de interes tiene 61.


In [ ]:
# Convertir el número de hogar como index en el marco de datos
df_EPFhogarPeninsula_.set_index('NUMERO')

,ANOENC,CCAA,CAPROV,TAMAMU,DENSIDAD,FACTOR,NMIEMB,TAMANO,NMIEM1,NMIEM2,...,NHABIT,ANNOCON,SUPERF,DISPOSIOV,CAPROP,CAJENA,PENSIO,RENTAS,IMPEXAC,NUMPERI
NUMERO,,,,,,,,,,,,,,,,,,,,,
1,2021,8,6,3,2,585.776970,1,1,1,0,...,5,1,90,6,6,6,1,6,782,1
2,2021,3,6,1,1,627.355782,1,1,1,0,...,4,6,65,6,6,6,1,6,1283,1
4,2021,11,6,3,2,605.490446,1,1,1,0,...,4,6,70,6,6,6,1,6,1350,1
5,2021,10,1,1,1,1748.502373,1,1,1,0,...,4,6,76,6,6,6,1,6,1206,1
6,2021,16,1,1,1,470.264063,4,4,4,0,...,6,1,115,6,6,1,6,6,7774,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19390,2021,11,6,5,3,491.129543,5,5,3,2,...,5,6,100,6,1,6,6,1,828,1
19391,2021,8,6,4,2,627.336739,2,2,2,0,...,7,6,90,6,6,6,1,6,1188,1
19392,2021,9,6,3,2,1016.009331,1,1,1,0,...,4,6,100,1,6,6,1,6,1761,1


## Tabla de información de los miembros del hogar: **De interés**

In [ ]:
cols_mhogra_interes = ['ANOENC', 'NUMERO', 'NORDEN', 'SUSPRIN', 'EDAD',
       'SEXO', 'PAISNACIM', 'NACIONA', 'PAISNACION', 'SITURES', 'ECIVILLEGAL', 'FACTOR']

In [ ]:
df_EPFmhogar_pre = pd.read_csv(f'{DIR_DATASETS}/{CARPETA_DATASETS}/{sub_carpeta_DATASET_mHOGAR}/CSV/{NOMBRE_DATASET_mHOGAR}', sep='\t' , usecols=cols_mhogra_interes)

In [ ]:
df_EPFmHogar_Peninsula_ = (df_EPFmhogar_pre.query("NUMERO in @hogares_peninsula_"))

In [ ]:
len(df_EPFmHogar_Peninsula_.NUMERO.unique())

17835

Identificación del sustentador principal

In [ ]:
df_EPFmHogar_Peninsula_test = df_EPFmHogar_Peninsula_.loc[df_EPFmHogar_Peninsula_['SUSPRIN'] == 1]

In [ ]:
df_EPFmHogar_Peninsula_test = df_EPFmHogar_Peninsula_test.copy()
df_EPFmHogar_Peninsula_test['_NORDEN-SUSPRIN'] = df_EPFmHogar_Peninsula_test['NORDEN']

Cambio de formato de long a wide

In [ ]:
df_EPFmHogar_Peninsula_ = df_EPFmHogar_Peninsula_.copy()
df_EPFmHogar_Peninsula_['NORDEN'] = df_EPFmHogar_Peninsula_['NORDEN'].astype(str)

In [ ]:
df_EPFmHogar_Peninsula_ = df_EPFmHogar_Peninsula_.pivot(index='NUMERO', columns='NORDEN', values=['EDAD', 'SEXO'])

In [ ]:
df_EPFmHogar_Peninsula_.columns = df_EPFmHogar_Peninsula_.columns.map('_'.join)

In [ ]:
df_EPFmHogar_Peninsula_.fillna(0, inplace = True)

In [ ]:
df_EPFmHogar_Peninsula_ = df_EPFmHogar_Peninsula_.astype(int)

In [ ]:
df_EPFmHogar_Peninsula_test = df_EPFmHogar_Peninsula_test[['NUMERO', '_NORDEN-SUSPRIN']]
df_EPFmHogar_Peninsula_test

,NUMERO,_NORDEN-SUSPRIN
0,1,1
1,2,1
4,4,1
5,5,1
6,6,1
...,...,...
49389,19390,1
49394,19391,1
49396,19392,1
49397,19393,1


Union de ambos dataframe (sexo y edad con sustentador principal)

In [ ]:
df_EPFmHogar_Peninsula_ = df_EPFmHogar_Peninsula_.merge(df_EPFmHogar_Peninsula_test, on = 'NUMERO')

In [ ]:
df_EPFmHogar_Peninsula_['_NORDEN-SUSPRIN'].value_counts()

1    15149
2     2507
3      145
4       25
5        7
6        1
8        1
Name: _NORDEN-SUSPRIN, dtype: int64

## Tabla final para familias con gasto en carburante

###  Unión de ambas tablas

Los datos de gastos se unieron con los datos sobre los hogares y sus características económicas y demográficas. Este fichero contuvo las 12.749 familias con gastos en carburante.

In [ ]:
# union ambas tablas
df_EPF_peninsula_ = pd.merge(df_EPFgastos_cocheprivado_2, df_EPFhogarPeninsula_, on="NUMERO", how="right", indicator=True)
df_EPF_peninsula_carburante =  pd.merge(df_EPFgastos_carburante_2, df_EPFhogarPeninsula_, on="NUMERO", how="left", indicator=True)
df_EPF_peninsula_coche = pd.merge(df_EPFgastos_cocheprivado_2, df_EPFhogarPeninsula_, on="NUMERO", how="left", indicator=True)

In [ ]:
# union con tabla informacion miembros del hogar
df_EPF_peninsula_ = pd.merge(df_EPF_peninsula_, df_EPFmHogar_Peninsula_, on="NUMERO", how="right", indicator=False)

In [ ]:
df_EPF_peninsula_.drop(columns = ['_merge' , 'SECTOR'], inplace = True)

In [ ]:
df_EPF_peninsula_.fillna(0, inplace = True)
df_EPF_peninsula_carburante.fillna(0, inplace = True)
df_EPF_peninsula_coche.fillna(0, inplace = True)

In [ ]:
df_EPF_peninsula_ = df_EPF_peninsula_.astype(int)

###  Variables derivadas

In [ ]:
#verificación del merge
df_EPF_peninsula_

,NUMERO,gasto_04123,gasto_04124,gasto_07111,gasto_07112,gasto_07120,gasto_07211,gasto_07212,gasto_07213,gasto_07221,...,SEXO_13,SEXO_2,SEXO_3,SEXO_4,SEXO_5,SEXO_6,SEXO_7,SEXO_8,SEXO_9,_NORDEN-SUSPRIN
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,6,0,0,0,0,0,0,0,0,44567,...,0,6,1,6,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17830,19390,0,0,0,0,0,0,0,86383,737135,...,0,6,1,6,6,0,0,0,0,1
17831,19391,0,0,0,0,0,0,0,0,0,...,0,6,0,0,0,0,0,0,0,1
17832,19392,0,0,0,0,0,0,0,0,1906164,...,0,0,0,0,0,0,0,0,0,1
17833,19393,0,0,28105347,0,0,0,0,0,1883212,...,0,6,1,1,0,0,0,0,0,1


In [ ]:
#Creacion de nueva columna de capital de provincia
df_EPF_peninsula_['_CAPROV_BOL'] = ['Si' if CAPROV == 1 else 'No' for CAPROV in df_EPF_peninsula_['CAPROV']]
df_EPF_peninsula_carburante['_CAPROV_BOL'] = ['Si' if CAPROV == 1 else 'No' for CAPROV in df_EPF_peninsula_carburante['CAPROV']]
df_EPF_peninsula_coche['_CAPROV_BOL'] = ['Si' if CAPROV == 1 else 'No' for CAPROV in df_EPF_peninsula_coche['CAPROV']]

In [ ]:
# funcion para definir niveles de renta
def niveles_renta(row):
    if row['IMPEXAC'] <= 1000:
        val = "Bajo"
    elif (row['IMPEXAC'] > 1000) &  (row['IMPEXAC'] <= 2000):
        val = "Medio"
    elif (row['IMPEXAC'] > 2000) &  (row['IMPEXAC'] <= 3000):
        val = "Medio-Alto"
    elif row['IMPEXAC'] > 5000:
        val = "Alto"
    else:
        val = "0"
    return val

In [ ]:
# creacion de categorias por niveles de renta
df_EPF_peninsula_['_Nivel_Renta'] = df_EPF_peninsula_.apply(niveles_renta, axis=1)
df_EPF_peninsula_carburante['_Nivel_Renta'] = df_EPF_peninsula_carburante.apply(niveles_renta, axis=1)
df_EPF_peninsula_coche['_Nivel_Renta'] = df_EPF_peninsula_coche.apply(niveles_renta, axis=1)

In [ ]:
hogares_frecuentes_list = df_EPFgastos_carburante.NUMERO.values.tolist()
hogares_ocasionales_list = list(set(df_EPFgastos_cocheprivado.NUMERO.values.tolist()).difference(df_EPFgastos_carburante.NUMERO.values.tolist()))
hogares_novehiculo_list = list(set(df_EPF_peninsula_.NUMERO.values.tolist()).difference(df_EPFgastos_cocheprivado.NUMERO.values.tolist()))

In [ ]:
# funcion para definir uso del vehiculo
def tipo_usuario(row):
    if row['NUMERO'] in hogares_frecuentes_list:
        val = "Frecuente"
    elif row['NUMERO'] in hogares_ocasionales_list:
        val = "Ocasional"
    elif row['NUMERO'] in hogares_novehiculo_list:
        val = "Sin coche"
    else:
        val = "N/A"
    return val

In [ ]:
df_EPF_peninsula_

,NUMERO,gasto_04123,gasto_04124,gasto_07111,gasto_07112,gasto_07120,gasto_07211,gasto_07212,gasto_07213,gasto_07221,...,SEXO_3,SEXO_4,SEXO_5,SEXO_6,SEXO_7,SEXO_8,SEXO_9,_NORDEN-SUSPRIN,_CAPROV_BOL,_Nivel_Renta
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,No,Bajo
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,No,Medio
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,No,Medio
3,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,Si,Medio
4,6,0,0,0,0,0,0,0,0,44567,...,1,6,0,0,0,0,0,1,Si,Alto
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17830,19390,0,0,0,0,0,0,0,86383,737135,...,1,6,6,0,0,0,0,1,No,Bajo
17831,19391,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,No,Medio
17832,19392,0,0,0,0,0,0,0,0,1906164,...,0,0,0,0,0,0,0,1,No,Medio
17833,19393,0,0,28105347,0,0,0,0,0,1883212,...,1,1,0,0,0,0,0,1,Si,0


In [ ]:
df_EPF_peninsula_.to_csv(f"EPF_{AÑO.value}_gastos_hogar_mhogar.csv")

Información de movilidad

In [ ]:
df_EPF_peninsula_['tipo_usuario'] = df_EPF_peninsula_.apply(lambda row: tipo_usuario(row), axis=1)

In [ ]:
df_info_movilidad = pd.read_csv(f'{DIR_DATASETS}/input keila_27sep.csv', encoding='latin-1')

In [ ]:
df_EPF_movilidad = df_EPF_peninsula_.merge(tabla_info_movilidad, on = 'CCAA')

In [ ]:
df_EPF_movilidad

,NUMERO,gasto_04123,gasto_04124,gasto_07111,gasto_07112,gasto_07120,gasto_07211,gasto_07212,gasto_07213,gasto_07221,...,SEXO_9,_NORDEN-SUSPRIN,_CAPROV_BOL,_Nivel_Renta,tipo_usuario,Descripción,% urbano,% interurbano RCE conv,% interurbano RCE gran capacidad,% interurbano resto
0,1,0,0,0,0,0,0,0,0,0,...,0,1,No,Bajo,Sin coche,Castilla-La Mancha,0.05,0.108,0.535,0.305
1,66,0,0,0,0,0,0,0,0,0,...,0,1,No,Bajo,Ocasional,Castilla-La Mancha,0.05,0.108,0.535,0.305
2,68,0,0,0,0,0,0,0,0,197786,...,0,1,No,Medio-Alto,Frecuente,Castilla-La Mancha,0.05,0.108,0.535,0.305
3,69,0,0,0,0,0,0,0,0,0,...,0,1,No,Bajo,Sin coche,Castilla-La Mancha,0.05,0.108,0.535,0.305
4,82,0,0,0,0,0,0,0,0,2127292,...,0,1,No,Medio,Frecuente,Castilla-La Mancha,0.05,0.108,0.535,0.305
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17830,19243,0,0,0,0,0,0,0,0,0,...,0,1,No,Medio,Ocasional,Cantabria,0.05,0.113,0.608,0.227
17831,19264,0,0,0,0,0,0,0,0,1079791,...,0,1,No,Medio-Alto,Frecuente,Cantabria,0.05,0.113,0.608,0.227
17832,19269,0,0,0,0,0,0,0,0,587169,...,0,1,No,0,Frecuente,Cantabria,0.05,0.113,0.608,0.227
17833,19275,0,0,0,0,0,0,0,0,425389,...,0,1,Si,Medio-Alto,Frecuente,Cantabria,0.05,0.113,0.608,0.227


In [ ]:
df_EPF_movilidad['_litros_consumidos'] = df_EPF_movilidad['_gasto_total_carbu_mes'] / 1.2982

In [ ]:
df_EPF_movilidad['_km_recorridos'] = df_EPF_movilidad['_litros_consumidos'] * (100/6)

In [ ]:
# Seleccion de la tarifa
tarifa_inter_RCE_conv = 0.03
tarifa_RCE_gran_capacidad = 0.03
tarifa_inter_resto = 0.03

In [ ]:
# Calculo renta poblacion
df_EPF_movilidad['_renta_poblacion'] = df_EPF_movilidad['IMPEXAC'] * df_EPF_movilidad['FACTOR_x']

In [ ]:
#Tarifa pago por uso poblacion

# Calculo de los centimos por tipo de vía
df_EPF_movilidad['_cts_kms_inter_RCE_Conv_pobl'] = tarifa_inter_RCE_conv * df_EPF_movilidad['% interurbano RCE conv'] * df_EPF_movilidad['_km_recorridos'] * df_EPF_movilidad.FACTOR_x
df_EPF_movilidad['_cts_km_inter_RCE_granC_pobl'] = tarifa_RCE_gran_capacidad *  df_EPF_movilidad['% interurbano RCE gran capacidad'] * df_EPF_movilidad['_km_recorridos'] * df_EPF_movilidad.FACTOR_x
df_EPF_movilidad['_cts_km_inter_resto_pobl'] = tarifa_inter_resto * df_EPF_movilidad['% interurbano resto'] * df_EPF_movilidad['_km_recorridos'] * df_EPF_movilidad.FACTOR_x

#Calculo del cobro total
df_EPF_movilidad['_cts_km_inter_total_pobl'] = df_EPF_movilidad['_cts_kms_inter_RCE_Conv_pobl'] + df_EPF_movilidad['_cts_km_inter_RCE_granC_pobl'] + df_EPF_movilidad['_cts_km_inter_resto_pobl']

#Renta menos el cobro
df_EPF_movilidad['_renta_pos_cobro_pago_uso_pobl'] = df_EPF_movilidad['_renta_poblacion'] - df_EPF_movilidad['_cts_km_inter_total_pobl']